In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from esutil.htm import HTM
from astropy.io import fits
import glob

plt.rc('figure', dpi=120)

In [2]:
if False:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
    from pyspark.sql import Row

    sc = SparkContext('local[*]')
    sc.setLogLevel("WARN")
else:
    from pysparkling import Context
    sc = Context()

#spark = SparkSession(sc)

In [6]:
from pyspark.sql import Row

In [3]:
example_filename = "ptf_catalogs/PTF_200912024227_c_p_scie_t100845_u011771826_f02_p101001_c09.ctlg"
example_file = fits.open(example_filename)
example_data = example_file[1].data

In [4]:
example_data.dtype

dtype((numpy.record, [('NUMBER', '>i4'), ('FLAGS', '>i2'), ('XWIN_IMAGE', '>f8'), ('YWIN_IMAGE', '>f8'), ('X_WORLD', '>f8'), ('Y_WORLD', '>f8'), ('XPEAK_IMAGE', '>i4'), ('YPEAK_IMAGE', '>i4'), ('ERRTHETAWIN_IMAGE', '>f4'), ('DELTAWIN_J2000', '>f8'), ('X2WIN_IMAGE', '>f8'), ('Y2WIN_IMAGE', '>f8'), ('XYWIN_IMAGE', '>f8'), ('AWIN_WORLD', '>f4'), ('BWIN_WORLD', '>f4'), ('MAG_ISO', '>f4'), ('MAGERR_ISO', '>f4'), ('MAG_AUTO', '>f4'), ('MAGERR_AUTO', '>f4'), ('MAG_ISOCOR', '>f4'), ('MAGERR_ISOCOR', '>f4'), ('MAG_APER', '>f4', (5,)), ('MAGERR_APER', '>f4', (5,)), ('MAG_PETRO', '>f4'), ('MAGERR_PETRO', '>f4'), ('MAG_BEST', '>f4'), ('MAGERR_BEST', '>f4'), ('MU_THRESHOLD', '>f4'), ('MU_MAX', '>f4'), ('BACKGROUND', '>f4'), ('THRESHOLD', '>f4'), ('ALPHAWIN_J2000', '>f8'), ('THETAWIN_IMAGE', '>f4'), ('THETAWIN_J2000', '>f4'), ('ELONGATION', '>f4'), ('ISOAREA_WORLD', '>f4'), ('ISOAREAF_WORLD', '>f4'), ('ISO0', '>i4'), ('ISO1', '>i4'), ('ISO2', '>i4'), ('ISO3', '>i4'), ('ISO4', '>i4'), ('ISO5', '>i4')

In [14]:
ra_field, dec_field = "ALPHAWIN_J2000", "DELTAWIN_J2000"

In [11]:
%%prun
def readPTFFile(filename):
    f = fits.open(filename)
    data = f[1].data
    field_names = list(set(data.names) - set(('MAG_APER', 'MAGERR_APER', 'FLUX_APER', 'FLUXERR_APER', 'FLUX_RADIUS')))
    placeholders = {'htm_id': 0, 'obj_id': 0}
    rows = [Row(**{field: row[field].item() for field in field_names}, **placeholders) for row in data]
    return rows

def add_htmid(object_tuple):
    htm_obj = HTM(depth=7)
    ra_field, dec_field = "ALPHAWIN_J2000", "DELTAWIN_J2000"
    
    ra = object_tuple[ra_field]
    dec = object_tuple[dec_field]
    htm_id = htm_obj.lookup_id(ra, dec)
    
    return htm_id.item()

def match_sources(input_tuple):
    htm_id, records = input_tuple
    
    ra_field, dec_field = "ALPHAWIN_J2000", "DELTAWIN_J2000"
    
    input_dict = {k: v for (k, v) in zip(range(len(records)), records)}
    
    ra = np.array([x[ra_field] for x in records])
    dec = np.array([x[dec_field] for x in records])

    tree = cKDTree(np.stack((ra, dec), axis=1))

    candidate_groups = []
    # First pass, just to get groupings
    for this_id in range(len(ra)):
        dists, idx = tree.query( (ra[this_id], dec[this_id]), k=15, distance_upper_bound=4/3600.0)
        sel, = np.where((dists < 3/3600.0) & (idx != this_id))
        this_group = [input_dict[this_id]]
        for n in idx[sel]:
            this_group.append((input_dict[n]))
        candidate_groups.append(this_group)
    
    # Second pass uses the mean position of the sources in each group, then re-matches
    # This potentially matches sources into multiple objects
    output_groups = []
    for candidate_group in candidate_groups:
        mean_ra = np.mean(np.array([x[1] for x in candidate_group]))
        mean_dec = np.mean(np.array([x[2] for x in candidate_group]))        
        dists, idx = tree.query( (mean_ra, mean_dec), k=15, distance_upper_bound=4/3600.0)
        sel, = np.where((dists < 3/3600.0))
        this_group = []
        for n in idx[sel]:
            this_group.append((input_dict[n]))
        output_groups.append(this_group)
    return output_groups

def apply_obj_id(input):
    iterator, obj_id = input
    return [Row(**values.asDict(), **{"obj_id": obj_id, "htm_id": add_htmid(values)}) for values in iterator]

filenames = glob.glob("ptf_catalogs/*ctlg")[:3]
grouped_records = sc.parallelize(filenames).flatMap(readPTFFile).keyBy(add_htmid).groupByKey()
matched_records = grouped_records.map(match_sources).flatMap(lambda x: x).zipWithUniqueId()
tagged_source_records = matched_records.flatMap(apply_obj_id).collect()

#field_names = list(set(data.names) - set(('MAG_APER', 'MAGERR_APER', 'FLUX_APER', 'FLUXERR_APER', 'FLUX_RADIUS')))
#sources_df = spark.createDataFrame(tagged_source_records, schema=("source_id", "ra", "dec"))

KeyboardInterrupt: 

Pandas version
-----------

In [15]:
import pandas
from astropy.table import Table

In [114]:
%%time

# This is a critical performance setting, since otherwise
# pandas will manually call the garbage collector way too much
# and that will dominate the runtime.
pandas.set_option('mode.chained_assignment', None)

def readPTFFile_pandas(filename):
    table = Table.read(filename)
    table.remove_columns(('MAG_APER', 'MAGERR_APER', 'FLUX_APER', 'FLUXERR_APER', 'FLUX_RADIUS'))
    df = table.to_pandas()
    return df

def split_table_by_htmid(table):
    # Return a set of tuples [ (htm_id, Table), ... ]
    htm_obj = HTM(depth=5)
    ra_field, dec_field = "ALPHAWIN_J2000", "DELTAWIN_J2000"
    
    ra = table[ra_field]
    dec = table[dec_field]
    htm_id = htm_obj.lookup_id(ra, dec)
    table['htm_id'] = htm_id
    htm_groups = table.groupby(htm_id)
    return htm_groups

def match_sources_pandas(input_tuple):
    htm_id, table = input_tuple
    
    ra_field, dec_field = "ALPHAWIN_J2000", "DELTAWIN_J2000"    
    ra = table[ra_field]
    dec = table[dec_field]

    tree = cKDTree(np.stack((np.array(ra), np.array(dec)), axis=1))
    print("Len ", len(ra))

    candidate_groups = []
    # First pass, just to get groupings
    for this_id in range(len(ra)):
        dists, idx = tree.query( (ra.iloc[this_id], dec.iloc[this_id]), k=15, distance_upper_bound=4/3600.0)
        sel, = np.where((dists < 3/3600.0) & (idx != this_id))
        mean_ra = np.mean(candidate_table[ra_field])
        mean_dec = np.mean(candidate_table[dec_field])        
        candidate_groups.append(idx[sel])
    
    # Second pass uses the mean position of the sources in each group, then re-matches
    # This potentially matches sources into multiple objects
    output_groups = []
    for candidate_group_idx in candidate_groups:
        candidate_table = table.iloc[candidate_group_idx]
        mean_ra = np.mean(candidate_table[ra_field])
        mean_dec = np.mean(candidate_table[dec_field])
        dists, idx = tree.query( (mean_ra, mean_dec), k=15, distance_upper_bound=4/3600.0)
        sel, = np.where((dists < 3/3600.0))
        output_groups.append(idx[sel])
        
    table['obj_id'] = 0
    for n, group in enumerate(output_groups):
        table['obj_id'].iloc[group] = htm_id*10000 + n

    return table

zeroValue = None
def seqFunc(a, b):
    if a is None:
        return b
    if b is None:
        return a
    return a.append(b)

filenames = glob.glob("ptf_catalogs/*ctlg")[:3]
split_records = sc.parallelize(filenames).map(readPTFFile_pandas).flatMap(split_table_by_htmid)

grouped_records = split_records.aggregateByKey(zeroValue, seqFunc, seqFunc)
print("Number of HTM cells: ", grouped_records.count())

matched_records = grouped_records.map(match_sources_pandas).collect()


Number of HTM cells:  3
Len  2841
Len  1038
Len  8753
 

In [ ]:
# 239 seconds for three files

In [113]:
3

3

In [109]:
grouped_records.count()

30

In [83]:
print(sum(len(x[1]) for x in grouped_records))
print(len(grouped_records))
print(len(np.unique([x[0] for x in grouped_records])))

27515
30
30


In [126]:
np.mean(f1[ra_field])
ra = f1[ra_field]
tree = cKDTree(np.stack((np.array(ra), np.array(ra)), axis=1))
dists, indices = tree.query( np.stack((ra, ra), axis=1), k=range(2,6), )
dists, indices

(array([[  1.34634114e-04,   2.28064392e-04,   2.86760228e-04,
           5.72861103e-04],
        [  5.54066038e-05,   8.57350378e-05,   9.07024027e-05,
           1.72857427e-04],
        [  1.33305391e-05,   1.03715084e-04,   1.54453634e-04,
           1.63173604e-04],
        ..., 
        [  4.59113699e-06,   1.04178061e-04,   2.28945420e-04,
           3.69189623e-04],
        [  1.48978776e-04,   2.56733980e-04,   2.92814947e-04,
           4.28704740e-04],
        [  1.39946119e-04,   2.29920207e-04,   2.83713474e-04,
           3.76529026e-04]]), array([[4219, 2617, 3604,  138],
        [3852,  305, 3919, 2524],
        [ 902, 4056, 3190, 2201],
        ..., 
        [ 278, 1974, 1078, 2122],
        [3722, 3003, 3416, 3576],
        [3450, 2094, 1556, 1830]]))

In [124]:
dest_idx, source_idx = np.meshgrid(np.arange(15), np.arange(len(ra)))

In [125]:
source_idx

array([[   0,    0,    0, ...,    0,    0,    0],
       [   1,    1,    1, ...,    1,    1,    1],
       [   2,    2,    2, ...,    2,    2,    2],
       ..., 
       [4347, 4347, 4347, ..., 4347, 4347, 4347],
       [4348, 4348, 4348, ..., 4348, 4348, 4348],
       [4349, 4349, 4349, ..., 4349, 4349, 4349]])

In [120]:
sel, = np.where(dists_flat < 3/3600.0)


ValueError: too many values to unpack (expected 1)

In [51]:
f1 = readPTFFile_pandas(filenames[0])
f2 = readPTFFile_pandas(filenames[1])

In [88]:
f1.loc[2]

NUMBER               3.000000e+00
FLAGS                0.000000e+00
XWIN_IMAGE           2.890227e+02
YWIN_IMAGE           1.119977e+01
X_WORLD              8.180505e+01
Y_WORLD              1.894336e+00
XPEAK_IMAGE          2.890000e+02
YPEAK_IMAGE          1.100000e+01
ERRTHETAWIN_IMAGE   -8.540615e+01
DELTAWIN_J2000       1.894322e+00
X2WIN_IMAGE          2.628784e-01
Y2WIN_IMAGE          3.317448e-01
XYWIN_IMAGE         -1.692092e-02
AWIN_WORLD           1.630356e-04
BWIN_WORLD           1.431539e-04
MAG_ISO             -1.061281e+01
MAGERR_ISO           3.537020e-02
MAG_AUTO            -1.061281e+01
MAGERR_AUTO          3.537020e-02
MAG_ISOCOR          -1.070244e+01
MAGERR_ISOCOR        4.134292e-02
MAG_PETRO           -1.068183e+01
MAGERR_PETRO         4.923702e-02
MAG_BEST            -1.061281e+01
MAGERR_BEST          3.537020e-02
MU_THRESHOLD        -5.641225e+00
MU_MAX              -9.194663e+00
BACKGROUND           2.217480e+04
THRESHOLD            1.838010e+02
ALPHAWIN_J2000

In [55]:
len(f1), len(f2)

(4350, 4245)

In [46]:
table = df
htm_obj = HTM(depth=7)
ra_field, dec_field = "ALPHAWIN_J2000", "DELTAWIN_J2000"

ra = table[ra_field]
dec = table[dec_field]
htm_id = htm_obj.lookup_id(ra, dec)
print(np.unique(htm_id))
htm_groups = table.groupby(htm_id)

#table.add_column()

[245941 245945 245946 245947 245948 245949 245951]


In [50]:
print([len(x[1]) for x in htm_groups])
print([x[0] for x in htm_groups])

[312, 210, 872, 1409, 1373, 8, 166]
[245941, 245945, 245946, 245947, 245948, 245949, 245951]


In [71]:
id, iterator = grouped_records[0]
x = [x for x in iterator]

In [72]:
row = x[0]

In [9]:
%prun
x = 1234 + 213

In [29]:
rec = example_data[5]

In [62]:
rec['FSDF'].item() or "junk"

KeyError: "Key 'FSDF' does not exist."

In [50]:
example_data.names

['NUMBER',
 'FLAGS',
 'XWIN_IMAGE',
 'YWIN_IMAGE',
 'X_WORLD',
 'Y_WORLD',
 'XPEAK_IMAGE',
 'YPEAK_IMAGE',
 'ERRTHETAWIN_IMAGE',
 'DELTAWIN_J2000',
 'X2WIN_IMAGE',
 'Y2WIN_IMAGE',
 'XYWIN_IMAGE',
 'AWIN_WORLD',
 'BWIN_WORLD',
 'MAG_ISO',
 'MAGERR_ISO',
 'MAG_AUTO',
 'MAGERR_AUTO',
 'MAG_ISOCOR',
 'MAGERR_ISOCOR',
 'MAG_APER',
 'MAGERR_APER',
 'MAG_PETRO',
 'MAGERR_PETRO',
 'MAG_BEST',
 'MAGERR_BEST',
 'MU_THRESHOLD',
 'MU_MAX',
 'BACKGROUND',
 'THRESHOLD',
 'ALPHAWIN_J2000',
 'THETAWIN_IMAGE',
 'THETAWIN_J2000',
 'ELONGATION',
 'ISOAREA_WORLD',
 'ISOAREAF_WORLD',
 'ISO0',
 'ISO1',
 'ISO2',
 'ISO3',
 'ISO4',
 'ISO5',
 'ISO6',
 'ISO7',
 'FWHM_IMAGE',
 'KRON_RADIUS',
 'PETRO_RADIUS',
 'CLASS_STAR',
 'FLUX_BEST',
 'FLUXERR_BEST',
 'FLUX_AUTO',
 'FLUXERR_AUTO',
 'FLUX_ISO',
 'FLUXERR_ISO',
 'FLUX_APER',
 'FLUXERR_APER',
 'X_IMAGE',
 'Y_IMAGE',
 'X2_IMAGE',
 'Y2_IMAGE',
 'XY_IMAGE',
 'THETA_IMAGE',
 'ERRAWIN_IMAGE',
 'ERRBWIN_IMAGE',
 'THETAWIN_WORLD',
 'ERRX2WIN_IMAGE',
 'ERRY2WIN_IMAGE',
 

In [37]:
rec_row = Row(**rec)
rec_row

TypeError: type object argument after ** must be a mapping, not FITS_record

In [42]:
type(rec.field("FLUX_AUTO"))

numpy.float32

In [46]:
len(rec.array)

4350

In [47]:
len(example_data)

4350

In [25]:
htm, iterator = recs[0]
x = [x for x in iterator]

In [28]:
x[0].dtype

AttributeError: 'FITS_record' object has no attribute 'dtype'